<a href="https://colab.research.google.com/github/codewithkate/Defense_Policy_Analysis/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perform K-Means clustering

Modified code from: [Dhiraj Kumar at NeptuneAI](https://neptune.ai/blog/customer-segmentation-using-machine-learning)

In [1]:
# Import required libraries

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

## Upload dataset from your local file system
Download the dataset used in the is analysis from [GitHub](https://github.com/codewithkate/Defense_Policy_Analysis).

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving eda_data.csv to eda_data.csv
User uploaded file "eda_data.csv" with length 806992 bytes


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['eda_data.csv']))
df.shape

(4113, 18)

## Inspect the data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Item Description-original          4113 non-null   object 
 1   Item Description - cleaned         4113 non-null   object 
 2   Transfer Authority                 4113 non-null   object 
 3   Fiscal Year of Request             4113 non-null   object 
 4   Implementing Agency                4113 non-null   object 
 5   Country (Transfer to)              4113 non-null   object 
 6   Status                             4113 non-null   object 
 7   Status Date                        4113 non-null   object 
 8   Qty Requested                      4113 non-null   int64  
 9   Qty Allocated                      4113 non-null   object 
 10  Qty Accepted                       4113 non-null   int64  
 11  Qty Rejected                       4113 non-null   int64

In [5]:
df.describe()

,Qty Requested,Qty Accepted,Qty Rejected,Total Delivered Qty,Total Acquisition Value,Total Current Value,Request to Status Duration (Days),Grant Pricing
count,4.113000e+03,4.113000e+03,4113.000000,4.113000e+03,4.112000e+03,4.111000e+03,4113.000000,4.113000e+03
mean,5.236023e+04,7.975120e+03,397.891077,1.729981e+04,6.449672e+06,1.027025e+06,462.840749,2.585325e+05
std,7.521134e+05,3.531358e+05,18274.542193,4.914239e+05,3.528174e+07,6.311961e+06,367.668584,1.412200e+06
min,-9.990000e+02,-9.990000e+02,-999.000000,-9.990000e+02,0.000000e+00,0.000000e+00,2.000000,0.000000e+00
25%,1.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.747725e+04,9.068250e+03,217.000000,1.716922e+03
50%,3.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.900740e+05,4.913820e+04,351.000000,8.477302e+03
75%,3.400000e+01,1.000000e+00,0.000000,1.000000e+00,1.154636e+06,2.531300e+05,637.000000,5.100000e+04
max,2.342582e+07,2.142558e+07,1000000.000000,2.142558e+07,9.573756e+08,1.809981e+08,2644.000000,3.657175e+07


## K-Means

In [6]:
# Define K-means model
kmeans_model = KMeans(init='k-means++', n_init=10, max_iter=400, random_state=42)

In [7]:
# Train the model
kmeans_model.fit(df[['Request to Status Duration (Days)','Qty Requested',
'Grant Pricing']])

KMeans(max_iter=400, n_init=10, random_state=42)

### Elbow Method
Identify the best k-value by measuring the sum of squares within each cluster.


In [8]:
# Create the K means model for different values of K
def try_different_clusters(K, data):

    cluster_values = list(range(1, K+1))
    inertias=[]

    for c in cluster_values:
        model = KMeans(n_clusters = c,init='k-means++',n_init=10,max_iter=400,random_state=42)
        model.fit(data)
        inertias.append(model.inertia_)

    return inertias

In [9]:
# Find output for k values between 1 to 12
outputs = try_different_clusters(12, df[['Request to Status Duration (Days)','Qty Requested',
'Grant Pricing']])
distances = pd.DataFrame({"clusters": list(range(1, 13)),"sum of squared distances": outputs})

In [10]:
# Finding optimal number of clusters k
figure = go.Figure()
figure.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),
                  xaxis_title="Number of clusters",
                  yaxis_title="Sum of squared distances",
                  title_text="Finding optimal number of clusters using elbow method")
figure.show()

**Optimal value = 5**

The above visualization is used to determine the optimal value form where the curve of the line bends, like an elbow. This bend is based on the sum of square distances of each point from its center (centroid). If we add more than the optimal value then the model will decrease in performance and efficiency, as this where the the reduction in distance begins to slow down.

In [11]:
# Re-Train K means model with k=5
kmeans_model_new = KMeans(n_clusters = 5,init='k-means++',n_init = 10,max_iter=400,random_state=42)

kmeans_model_new.fit_predict(df[['Request to Status Duration (Days)','Qty Requested',
'Grant Pricing']])

array([0, 0, 0, ..., 0, 0, 4], dtype=int32)

In [12]:
# Create data arrays
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data, cluster_centers, axis=1)
points

<ipython-input-12-ffdb0ff0bada>:3: RuntimeWarning:

overflow encountered in expm1



array([[2.74715975e+198,             inf,             inf,
        4.56922416e+002, 1.96160715e+004, 7.93710120e+004],
       [            inf,             inf,             inf,
        8.78333333e+002, 8.40000000e+002, 3.30462466e+007],
       [            inf, 5.22632645e+193,             inf,
        8.26052632e+002, 4.46052632e+002, 1.26086850e+007],
       [3.39914613e+223,             inf,             inf,
        5.14700000e+002, 1.37685045e+007, 1.77764403e+005],
       [4.77346614e+252, 7.33453646e+142,             inf,
        5.81814516e+002, 3.28959677e+002, 3.29670577e+006]])

In [13]:
# Add "clusters" to customers data
points = np.append(points, [[0], [1], [2], [3], [4]], axis=1)
df["clusters"] = kmeans_model_new.labels_

In [14]:
df[['clusters']].value_counts()

clusters
0           3957
4            124
2             19
3             10
1              3
dtype: int64

In [15]:
# visualize clusters
figure = px.scatter_3d(df,
                    color='clusters',
                    x='Request to Status Duration (Days)',
                    y='Qty Requested',
                    z='Grant Pricing',
                    category_orders = {"clusters": ["0", "1", "2", "3", "4"]}
                    )
figure.update_layout()
figure.show()

## Recommendations

*   Standardize quantitative values
*   Perform analysis on smaller, random sample
*   Include related business factors
*   Adjust KMeans parameters
*   Try other clustering methods, such as PAM, DBSCAN, or FCM



